# Lab 03 - Topic Modelling
In this lab we will look into building topic models, but will also examine dimensionality reduction and other relevant subjects.

### Latent Dirichlet Allocation (LDiA)

Based on: [Evaluate Topic Models: Latent Dirichlet Allocation (LDA)](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)

** **

For this lab, we’ll use the dataset of papers published in NeurIPS (NIPS) conference. The CSV data file contains information on the different NeurIPS papers that were published from 1987 until 2016. These papers discuss a wide variety of topics in machine learning, from neural networks to optimization methods, and many more.

** **
#### Step 1: Loading Data
** **

In [1]:
!wget https://www.cse.iitb.ac.in/~diptesh/data.tar.gz && tar -xvzf data.tar.gz

--2022-03-01 12:34:24--  https://www.cse.iitb.ac.in/~diptesh/data.tar.gz
Resolving www.cse.iitb.ac.in (www.cse.iitb.ac.in)... 103.21.127.134
Connecting to www.cse.iitb.ac.in (www.cse.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73485589 (70M) [application/x-gzip]
Saving to: 'data.tar.gz'

data.tar.gz         100%[===================>]  70.08M  12.3MB/s    in 7.4s    

2022-03-01 12:34:32 (9.49 MB/s) - 'data.tar.gz' saved [73485589/73485589]

data/
data/papers.csv
data/.DS_Store
data/articles+4.txt
data/authors.csv
data/paper_authors.csv


In [2]:
# Importing modules
import pandas as pd
import os

#os.chdir('..')

# Read data into papers
papers = pd.read_csv('data/papers.csv')

# Print head
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


** **
#### Step 2: Data Cleaning
** **

Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each paper, and drop other metadata columns

In [2]:
# Remove the columns
papers = papers.drop(columns=['id', 'title', 'abstract', 
                              'event_type', 'pdf_name', 'year'], axis=1)

# sample only 100 papers
papers = papers.sample(100)

# Print out the first rows of papers
papers.head()

,paper_text
6355,830\n\nInvariant Object Recognition Using a Di...
4748,338\n\nThe Connectivity Analysis of Simple Ass...
6206,An Information-Theoretic Approach to\nDecipher...
4830,Mind the Nuisance: Gaussian Process\nClassific...
2045,Integrate-and-Fire models with adaptation are\...


##### Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of paper_text column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [3]:
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

6355    830\n\ninvariant object recognition using a di...
4748    338\n\nthe connectivity analysis of simple ass...
6206    an information-theoretic approach to\ndecipher...
4830    mind the nuisance: gaussian process\nclassific...
2045    integrate-and-fire models with adaptation are\...
Name: paper_text_processed, dtype: object

##### Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [4]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['invariant', 'object', 'recognition', 'using', 'distributed', 'associative', 'memory', 'harry', 'wechsler', 'and', 'george', 'lee', 'zimmerman', 'department', 'or', 'electrical', 'engineering', 'university', 'or', 'minnesota', 'minneapolis', 'mn', 'abstract', 'this', 'paper', 'describes', 'an', 'approach', 'to', 'dimensional']


** **
#### Step 3: Phrase Modeling: Bigram and Trigram Models
** **

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring. Some examples in our example are: 'back_bumper', 'oil_leakage', 'maryland_college_park' etc.

Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

In [5]:
%%time
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

CPU times: user 5.3 s, sys: 59.4 ms, total: 5.36 s
Wall time: 6.2 s


#### Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [6]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vrusiasb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let's call the functions in order.

In [8]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7 MB 12.8 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['invariant', 'object', 'recognition', 'use', 'distributed_associative', 'memory', 'harry', 'wechsl', 'george', 'lee', 'zimmerman', 'department', 'electrical_engineering', 'university', 'minnesota', 'minneapolis', 'abstract', 'paper', 'describe', 'approach', 'dimensional', 'object', 'recognition', 'complex', 'log', 'conformal', 'mapping', 'combine', 'distributed_associative', 'memory']


** **
#### Step 4: Data transformation: Corpus and Dictionary
** **

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [10]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 3), (2, 4), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 3), (16, 4), (17, 1), (18, 5), (19, 2), (20, 4), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 4), (27, 2), (28, 1), (29, 1)]


** **
#### Step 5: Base Model 
** **

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [11]:
%%time
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

CPU times: user 22.7 s, sys: 3.35 s, total: 26 s
Wall time: 10.7 s


** **
The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()`

In [12]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"model" + 0.009*"set" + 0.008*"image" + 0.008*"use" + 0.006*"datum" + '
  '0.006*"face" + 0.005*"function" + 0.005*"learn" + 0.005*"time" + '
  '0.005*"algorithm"'),
 (1,
  '0.011*"model" + 0.009*"sample" + 0.009*"distribution" + 0.008*"use" + '
  '0.008*"test" + 0.007*"datum" + 0.007*"set" + 0.006*"function" + '
  '0.006*"result" + 0.006*"point"'),
 (2,
  '0.013*"model" + 0.009*"use" + 0.008*"state" + 0.007*"learn" + 0.007*"time" '
  '+ 0.006*"datum" + 0.006*"function" + 0.006*"policy" + 0.006*"show" + '
  '0.005*"local"'),
 (3,
  '0.011*"function" + 0.009*"algorithm" + 0.008*"use" + 0.007*"set" + '
  '0.007*"sample" + 0.007*"error" + 0.006*"learn" + 0.006*"problem" + '
  '0.006*"theorem" + 0.006*"distribution"'),
 (4,
  '0.009*"distribution" + 0.008*"function" + 0.008*"learn" + 0.007*"method" + '
  '0.007*"feature" + 0.006*"show" + 0.006*"result" + 0.006*"set" + '
  '0.005*"problem" + 0.005*"variable"'),
 (5,
  '0.015*"function" + 0.012*"network" + 0.011*"algorithm" + 0

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [14]:
%%time
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.27588331858083587
CPU times: user 582 ms, sys: 138 ms, total: 720 ms
Wall time: 15.3 s


** **
#### Step 6: Hyperparameter tuning
#### Do not do that during the labs as it will take a long time! So skip to Step 7 for now
** **
First, let's differentiate between model hyperparameters and model parameters :

- `Model hyperparameters` can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

- `Model parameters` can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters: 
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use `C_v` as our choice of metric for performance comparison 

In [14]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [ ]:
%%time
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../results/lda_tuning_results.csv', index=False)
    pbar.close()

 28%|██▊       | 151/540 [39:55<1:57:47, 18.17s/it]

** **
#### Step 7: Final Model
** **

Based on external evaluation (Code to be added from Excel based analysis), let's train the final model with parameters yielding highest coherence score

In [15]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [16]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"model" + 0.008*"use" + 0.007*"datum" + 0.007*"set" + '
  '0.007*"function" + 0.007*"image" + 0.007*"learn" + 0.005*"result" + '
  '0.005*"class" + 0.005*"learning"'),
 (1,
  '0.005*"function" + 0.005*"model" + 0.005*"distribution" + 0.004*"result" + '
  '0.004*"datum" + 0.004*"sample" + 0.004*"use" + 0.003*"algorithm" + '
  '0.003*"method" + 0.003*"show"'),
 (2,
  '0.011*"model" + 0.007*"use" + 0.006*"time" + 0.006*"state" + '
  '0.006*"distribution" + 0.006*"algorithm" + 0.005*"policy" + 0.005*"cell" + '
  '0.005*"function" + 0.005*"learn"'),
 (3,
  '0.008*"algorithm" + 0.006*"function" + 0.006*"problem" + 0.005*"use" + '
  '0.005*"sample" + 0.004*"learn" + 0.004*"model" + 0.004*"set" + '
  '0.004*"error" + 0.004*"vector"'),
 (4,
  '0.004*"crf" + 0.004*"relaxation" + 0.003*"message" + 0.003*"variable" + '
  '0.002*"potential" + 0.002*"inference" + 0.002*"define" + 0.002*"factor" + '
  '0.002*"estimation" + 0.002*"message_estimator"'),
 (5,
  '0.009*"network" + 0.008*"fu

** **
#### Step 8: Visualize Results
** **

In [20]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('../results/ldavis_tuned_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, '../results/ldavis_tuned_'+ str(num_topics) +'.html')

LDAvis_prepared

/opt/anaconda3/envs/nlp2021-2/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.087639  0.026779       1        1  31.393005
2      0.055192 -0.027987       2        1  20.235450
3      0.026125  0.004406       3        1  12.957076
5      0.018279 -0.053168       4        1  12.461287
1      0.004910  0.017987       5        1  11.176085
6     -0.034666  0.044608       6        1   5.532799
7     -0.045516  0.001491       7        1   4.619842
4     -0.111962 -0.014116       8        1   1.624456, topic_info=         Term        Freq       Total Category  logprob  loglift
418   network  518.000000  518.000000  Default  30.0000  30.0000
305     image  569.000000  569.000000  Default  29.0000  29.0000
429    object  202.000000  202.000000  Default  28.0000  28.0000
2448   policy  320.000000  320.000000  Default  27.0000  27.0000
82       cell  249.000000  249.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
2509   target    4.126839  162.269700   Topic8  -6.6746   0.4482
946     learn    4.708374  887.372944   Topic8  -6.5427  -1.1189
690    vector    4.299822  385.879587   Topic8  -6.6335  -0.3770
1442    spike    3.890292  156.256060   Topic8  -6.7336   0.4270
573       set    4.133838  959.519700   Topic8  -6.6729  -1.3272

[558 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
716       1  0.435704  action
716       2  0.458338  action
716       3  0.011317  action
716       4  0.079219  action
716       5  0.005658  action
...     ...       ...     ...
5073      1  0.064087      zd
5073      2  0.064087      zd
5073      3  0.769042      zd
5073      4  0.064087      zd
5073      5  0.064087      zd

[2538 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 6, 2, 7, 8, 5])

** **
#### Closing Notes

We started with understanding why evaluating the topic model is essential. Next, we reviewed existing methods and scratched the surface of topic coherence, along with the available coherence measures. Then we built a default LDA model using Gensim implementation to establish the baseline coherence score and reviewed practical ways to optimize the LDA hyperparameters.

Hopefully, this article has managed to shed light on the underlying topic evaluation strategies, and intuitions behind it.

** **
#### References:
1. http://qpleple.com/perplexity-to-evaluate-topic-models/
2. https://www.amazon.com/Machine-Learning-Probabilistic-Perspective-Computation/dp/0262018020
3. https://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf
4. https://github.com/mattilyra/pydataberlin-2017/blob/master/notebook/EvaluatingUnsupervisedModels.ipynb
5. https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
6. http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf
7. http://palmetto.aksw.org/palmetto-webapp/